In [56]:
import pandas as pd
import requests
from pathlib import Path
import json
import datetime

import numpy as np

import load_data as ld
import os

import twitter_lib as tl
import wsj_lib as wsj

In [57]:
coin = 'ethereum'
coin_ticker='ETH-USD'
num_of_periods = 1000 # 1000 days

start="2018-04-01T00:00:00Z"
end ="2022-04-30T00:00:00Z"
#default bin size of 24H, but we will provide ...just..
bin_size='24H'

In [58]:
#sentiment_df = tl.get_crypto_sentiment_from_twitter(coin, num_of_periods, start, end, bin_size)

In [59]:
#bear_sentiment

In [60]:
#bull_sentiment

In [61]:
#sentiment_df.head()

In [62]:
wsj_sentiment=wsj.get_sentiments_from_wsj()

In [63]:
wsj_sentiment.index

Index(['1/1/00', '1/1/01', '1/1/02', '1/1/03', '1/1/04', '1/1/06', '1/1/07',
       '1/1/08', '1/1/09', '1/1/10',
       ...
       '9/9/11', '9/9/12', '9/9/13', '9/9/15', '9/9/16', '9/9/17', '9/9/18',
       '9/9/19', '9/9/20', '9/9/98'],
      dtype='object', name='Date', length=7977)

In [64]:
#sentiment_df = pd.merge(bear_sentiment['Bear_sum'], bull_sentiment['Bull_sum'], how='inner', left_on='Date', right_on='Date')
#sentiment_df = pd.merge(bear_sentiment, bull_sentiment, how='inner', left_on='Date', right_on='Date')

In [65]:
#sentiment_df=sentiment_df.reset_index()

In [66]:
#sentiment_df['Date']=pd.to_datetime(sentiment_df['Date'])

In [67]:
#df_cd = pd.merge(df_SN7577i_c, df_SN7577i_d, how='inner', left_on = 'Id', right_on = 'Id')
#sentiment_df = pd.merge(sentiment_df[['Bear_sum','Bull_sum']], wsj_sentiment['sentiment'], how='inner', left_on='Date', right_on='Date')

In [68]:
#sentiment_df.set_index('Date')

In [69]:
wsj_sentiment=wsj_sentiment.reset_index()

In [70]:
wsj_sentiment['Date']=pd.to_datetime(wsj_sentiment['Date'])

In [71]:
wsj_sentiment=wsj_sentiment.set_index('Date')

In [72]:
wsj_sentiment=wsj_sentiment.groupby(by='Date').mean()

In [73]:
wsj_sentiment

,scores
Date,
1998-01-07,-0.273200
1998-02-12,0.000000
1998-03-26,0.000000
1998-05-03,0.000000
1998-05-04,0.000000
...,...
2021-06-26,0.152933
2021-06-27,0.000000
2021-06-28,-0.273200


In [74]:
#sentiment_df = pd.merge(sentiment_df, wsj_sentiment, how='inner', left_on='Date', right_on='Date')

In [75]:
#sentiment_df = sentiment_df.set_index('Date')

In [76]:
#ONLY WSJ in the SENTIMENT ANALYSIS
sentiment_df = wsj_sentiment.copy()

In [77]:
df_coin= ld.load_from_yahoo([coin_ticker], start,end)

In [78]:
'''short_window=10
long_window = 100
df_coins=pd.DataFrame(df_coin[coin_ticker]['close'])
df_coins['sig']=0.0
df_coins['SMA_short'] = df_coins['close'].rolling(window=short_window).mean()
df_coins['SMA_long'] = df_coins['close'].rolling(window=long_window).mean()
df_coins['sig'][short_window:] = np.where(
    df_coins['SMA_short'][short_window:] > df_coins['SMA_long'][short_window:], 1.0, 0.0
    )
df_coins['signal'] = df_coins['sig'].diff()
    
df_coins[df_coins['signal']==-1]
'''

"short_window=10\nlong_window = 100\ndf_coins=pd.DataFrame(df_coin[coin_ticker]['close'])\ndf_coins['sig']=0.0\ndf_coins['SMA_short'] = df_coins['close'].rolling(window=short_window).mean()\ndf_coins['SMA_long'] = df_coins['close'].rolling(window=long_window).mean()\ndf_coins['sig'][short_window:] = np.where(\n    df_coins['SMA_short'][short_window:] > df_coins['SMA_long'][short_window:], 1.0, 0.0\n    )\ndf_coins['signal'] = df_coins['sig'].diff()\n    \ndf_coins[df_coins['signal']==-1]\n"

In [79]:
df_coin.index

DatetimeIndex(['2018-03-31', '2018-04-01', '2018-04-02', '2018-04-03',
               '2018-04-04', '2018-04-05', '2018-04-06', '2018-04-07',
               '2018-04-08', '2018-04-09',
               ...
               '2022-04-21', '2022-04-22', '2022-04-23', '2022-04-24',
               '2022-04-25', '2022-04-26', '2022-04-27', '2022-04-28',
               '2022-04-29', '2022-04-30'],
              dtype='datetime64[ns]', name='Date', length=1492, freq=None)

In [80]:
joined_df = sentiment_df.join(df_coin[coin_ticker]['close'])

In [81]:
joined_df['daily_ret']=joined_df['close'].pct_change().shift()

In [82]:
joined_df

,scores,close,daily_ret
Date,,,
1998-01-07,-0.273200,NaN,NaN
1998-02-12,0.000000,NaN,NaN
1998-03-26,0.000000,NaN,NaN
1998-05-03,0.000000,NaN,NaN
1998-05-04,0.000000,NaN,NaN
...,...,...,...
2021-06-26,0.152933,1829.239258,-0.088128
2021-06-27,0.000000,1978.894653,0.008836
2021-06-28,-0.273200,2079.657471,0.081813


In [83]:
'''#polarity=[]
#subj=[]
signal=[]
cnt=True
#for i in f_df['Headline']:
for i in joined_df.index:
    #Bear_sum	Bull_sum	scores	
    if cnt : 
        i_previous = i
        signal.append(0)
        cnt = False
        continue
    if (joined_df['daily_ret'][i] >0 and joined_df['daily_ret'][i_previous] > 0) :
        signal.append(1)
    elif (joined_df['daily_ret'][i] < 0 and joined_df['daily_ret'][i_previous] < 0):
        signal.append(-1)
    else:
        signal.append(0)
    
    i_previous = i

'''

"#polarity=[]\n#subj=[]\nsignal=[]\ncnt=True\n#for i in f_df['Headline']:\nfor i in joined_df.index:\n    #Bear_sum\tBull_sum\tscores\t\n    if cnt : \n        i_previous = i\n        signal.append(0)\n        cnt = False\n        continue\n    if (joined_df['daily_ret'][i] >0 and joined_df['daily_ret'][i_previous] > 0) :\n        signal.append(1)\n    elif (joined_df['daily_ret'][i] < 0 and joined_df['daily_ret'][i_previous] < 0):\n        signal.append(-1)\n    else:\n        signal.append(0)\n    \n    i_previous = i\n\n"

In [84]:
only_price_change_signal=True

In [85]:
#USE DIFF FUNCTION TO DECIDE SIGNAL
if only_price_change_signal:
    joined_df['Diff'] = joined_df['daily_ret'].diff()
    joined_df['signal'] = joined_df['Diff'].apply(lambda x: 1 if x > 0.07 else -1 if x < 0.03 else 0)
    joined_df.drop(columns=['Diff'], inplace=True)
else:
    signal=[]
    #for i in f_df['Headline']:
    for i in joined_df.index:
        if (joined_df['scores'][i] > .5) and joined_df['daily_ret'][i] >0:
            signal.append(1)
        elif (joined_df['scores'][i]>0.0 and joined_df['scores'][i] <= .5) and joined_df['daily_ret'][i] < 0:
            signal.append(-1)
        else:
            signal.append(0)
    joined_df['signal']= signal

In [86]:
joined_df['signal']

Date
1998-01-07    0
1998-02-12    0
1998-03-26    0
1998-05-03    0
1998-05-04    0
             ..
2021-06-26   -1
2021-06-27    1
2021-06-28    1
2021-06-29   -1
2021-06-30   -1
Name: signal, Length: 7977, dtype: int64

In [87]:
joined_df

,scores,close,daily_ret,signal
Date,,,,
1998-01-07,-0.273200,NaN,NaN,0
1998-02-12,0.000000,NaN,NaN,0
1998-03-26,0.000000,NaN,NaN,0
1998-05-03,0.000000,NaN,NaN,0
1998-05-04,0.000000,NaN,NaN,0
...,...,...,...,...
2021-06-26,0.152933,1829.239258,-0.088128,-1
2021-06-27,0.000000,1978.894653,0.008836,1
2021-06-28,-0.273200,2079.657471,0.081813,1


In [88]:
joined_df=joined_df.dropna()

In [89]:
joined_df

,scores,close,daily_ret,signal
Date,,,,
2018-04-02,-0.045900,386.424988,-0.042501,0
2018-04-03,-0.458800,416.893005,0.017961,0
2018-04-04,0.000000,380.542999,0.078846,0
2018-04-05,-0.309633,383.231995,-0.087193,-1
2018-04-06,-0.007550,370.285004,0.007066,1
...,...,...,...,...
2021-06-26,0.152933,1829.239258,-0.088128,-1
2021-06-27,0.000000,1978.894653,0.008836,1
2021-06-28,-0.273200,2079.657471,0.081813,1


In [90]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression


In [91]:
y = joined_df['signal']
if only_price_change_signal:
    X = joined_df.drop(columns=['daily_ret', 'close', 'signal'])
else:
    X = joined_df.drop(columns=['scores', 'daily_ret', 'close', 'signal'])

In [92]:
X.to_csv('X.csv')
y.to_csv('y.csv')

In [93]:
X_train, X_test, y_train, y_test = train_test_split(X, y)
X_train

,scores
Date,
2018-12-16,-0.142867
2019-04-07,0.113333
2021-01-24,-0.299700
2020-03-11,-0.021200
2020-08-04,-0.170000
...,...
2020-03-04,0.263350
2021-05-24,0.059300
2020-02-24,0.139350


In [94]:
# Scale the data
scaler = StandardScaler()
X_scaler = scaler.fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [95]:
X_train_scaled.shape

(856, 1)

In [96]:
# Import ClusterCentroids from imblearn
from imblearn.under_sampling import ClusterCentroids

# Instantiate a ClusterCentroids instance
cc_sampler = ClusterCentroids(random_state=1)
# Fit the training data to the cluster centroids model
X_resampled, y_resampled = cc_sampler.fit_resample(X_train_scaled, y_train)

/Users/ashokpandey/opt/anaconda3/envs/dev/lib/python3.7/site-packages/imblearn/under_sampling/_prototype_generation/_cluster_centroids.py:172: ConvergenceWarning: Number of distinct clusters (117) found smaller than n_clusters (118). Possibly due to duplicate points in X.
  self.estimator_.fit(X[y == target_class])


In [97]:
# Count distinct values for the resampled target data
y_resampled.value_counts()

-1    118
 0    118
 1    118
Name: signal, dtype: int64

In [98]:
lr = LogisticRegression()

In [99]:
lr.fit(X_resampled, y_resampled)
# Make predictions using the test data
y_pred = lr.predict(X_test_scaled)

results = pd.DataFrame({
    "Prediction": y_pred, 
    "Actual": y_test
}).reset_index(drop=True)

results.head()


,Prediction,Actual
0,1,-1
1,0,-1
2,1,-1
3,1,0
4,1,-1


In [100]:
# Make predictions using the test data
y_pred = lr.predict(X_test_scaled)

results = pd.DataFrame({
    "Prediction": y_pred, 
    "Actual": y_test
}).reset_index(drop=True)

results.head()

,Prediction,Actual
0,1,-1
1,0,-1
2,1,-1
3,1,0
4,1,-1


In [101]:
# Import Modules
from sklearn.metrics import confusion_matrix,classification_report

In [102]:
# Print confusion matrix
print(confusion_matrix(y_test, y_pred))

[[  4 111  94]
 [  1  22  14]
 [  2  27  11]]


In [103]:
# Print classification report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

          -1       0.57      0.02      0.04       209
           0       0.14      0.59      0.22        37
           1       0.09      0.28      0.14        40

    accuracy                           0.13       286
   macro avg       0.27      0.30      0.13       286
weighted avg       0.45      0.13      0.08       286



In [104]:
#get the current news and data from augmento
#run the data prep to get the X_test_scaled data so it can be tested


In [105]:
# Instantiate a linear SVM model
from sklearn.svm import SVC
svm = SVC(kernel='linear')
# Fit the data
svm.fit(X_resampled, y_resampled)
# Make predictions using the test data
y_pred = svm.predict(X_test_scaled)

results = pd.DataFrame({
    "Prediction": y_pred, 
    "Actual": y_test
}).reset_index(drop=True)

results.head()
# Print classification report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

          -1       0.00      0.00      0.00       209
           0       0.14      0.59      0.22        37
           1       0.10      0.33      0.16        40

    accuracy                           0.12       286
   macro avg       0.08      0.31      0.13       286
weighted avg       0.03      0.12      0.05       286



/Users/ashokpandey/opt/anaconda3/envs/dev/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/ashokpandey/opt/anaconda3/envs/dev/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/ashokpandey/opt/anaconda3/envs/dev/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier

In [106]:
'''
c_rep=[]
c_rep.append(classification_report(y_test, y_pred, output_dict=True))
c_rep[0]['Name']="twitter"
c_rep[0]
'''

'\nc_rep=[]\nc_rep.append(classification_report(y_test, y_pred, output_dict=True))\nc_rep[0][\'Name\']="twitter"\nc_rep[0]\n'

In [107]:
#c_df.transpose

In [108]:
print('I m done..')

I m done..


In [53]:
import os
os.getcwd()

'/Users/ashokpandey/Desktop/Fintech-Workspace/Project 2/r&d_ashok'

In [54]:
X.to_csv('X_eth.csv')